In [5]:
import json
import os
import dashscope
from dashscope.api_entities.dashscope_response import Role
from dotenv import load_dotenv

load_dotenv()  # 加载 .env 文件
api_key = os.environ.get("DASHSCOPE_API_KEY")

dashscope.api_key = api_key

In [6]:
# 编写你的函数
def get_current_weather(location, unit="摄氏度"):
    # 获取指定地点的天气
    temperature = -1
    if '大连' in location or 'Dalian' in location:
        temperature = 10
    if location=='上海' or 'Shanghai' in location:
        temperature = 36
    if location=='深圳' or 'Shenzhen' in location:
        temperature = 37
    weather_info = {
        "location": location,
        "temperature": temperature,
        "unit": unit,
        "forecast": ["晴天", "微风"],
    }
    return json.dumps(weather_info)

functions = [
    {
      'name': 'get_current_weather',
      'description': '获取指定城市的天气情况.',
      'parameters': {
            'type': 'object',
            'properties': {
                'location': {
                    'type': 'string',
                    'description': '城市名称，如上海'
                },
                'unit': {'type': 'string', 'enum': ['celsius', 'fahrenheit']}
            },
        'required': ['location']
      }
    }
]

In [7]:
# 封装模型响应函数
def get_response(messages):
    try:
        response = dashscope.Generation.call(
            model='qwen-turbo',
            messages=messages,
            functions=functions,
            result_format='message'
        )
        return response
    except Exception as e:
        print(f"API调用出错: {str(e)}")
        return None

In [15]:
# 使用function call进行QA
from typing import Dict, Any, Optional
def run_conversation(user_query: str) -> Optional[Dict[str, Any]]:
    messages=[{"role": "user", "content": user_query}]
    
    # 得到第一次响应
    response = get_response(messages)
    if not response or not response.output:
        print("获取响应失败")
        return None
        
    print('response=', response)
    
    message = response.output.choices[0].message
    messages.append(message)
    print('message=', message)
    
    # Step 2, 判断用户是否要call function
    if hasattr(message, 'function_call') and message.function_call:
        function_call = message.function_call
        tool_name = function_call['name']
        # Step 3, 执行function call
        arguments = json.loads(function_call['arguments'])
        print('arguments=', arguments)
        # Step 3: 执行函数调用
        tool_response = f"Error: 函数执行失败"
        if tool_name == "get_current_weather":
                tool_response = get_current_weather(
                    location=arguments.get("location"),
                    unit=arguments.get("unit", "celsius"),
            )
        tool_info = {"role": "function", "name": tool_name, "content": tool_response}
        print('tool_info=', tool_info)
        messages.append(tool_info)
        print('messages=', messages)
        
        #Step 4, 得到第二次响应
        response = get_response(messages)
        if not response or not response.output:
            print("获取第二次响应失败")
            return None
            
        print('response=', response)
        message = response.output.choices[0].message
        return message
    return message


In [16]:
if __name__ == "__main__":
    result = run_conversation("大连的天气怎样")
    if result:
        print("最终结果:", result)
    else:
        print("对话执行失败")

response= {"status_code": 200, "request_id": "ca208306-f8bf-90ce-907d-a068c396aae4", "code": "", "message": "", "output": {"text": null, "finish_reason": null, "choices": [{"finish_reason": "function_call", "message": {"role": "assistant", "content": "", "function_call": {"name": "get_current_weather", "arguments": "{\"location\": \"大连\"}"}}}]}, "usage": {"input_tokens": 196, "output_tokens": 18, "total_tokens": 214, "prompt_tokens_details": {"cached_tokens": 0}}}
message= {"role": "assistant", "content": "", "function_call": {"name": "get_current_weather", "arguments": "{\"location\": \"大连\"}"}}
arguments= {'location': '大连'}
tool_info= {'role': 'function', 'name': 'get_current_weather', 'content': '{"location": "\\u5927\\u8fde", "temperature": 10, "unit": "celsius", "forecast": ["\\u6674\\u5929", "\\u5fae\\u98ce"]}'}
messages= [{'role': 'user', 'content': '大连的天气怎样'}, Message({'role': 'assistant', 'content': '', 'function_call': {'name': 'get_current_weather', 'arguments': '{"location"